# Experiment: What does Canadian History Look Like To the Machine?

If we generate say 100 images with https://www.craiyon.com/ or https://perchance.org/ai-text-to-image-generator and use the same prompt each time - a prompt related to canadian history - and we try the '[datafication of a kiss](https://www.cyberneticforests.com/news/how-to-read-an-ai-image)' approach[^1], what can we see? The idea is to treat the generated images as an infographic of the underlying dataset.

Go to craiyon or perchance (perchance seems to have fewer hidden guardrails); use the same prompt 11 times. Take a screenshot of the results each time (the image grid that Craiyon returns). Rename your screenshots 1, 2, 3, 4, 5 etc. Drag and drop the screenshots into the file tray here. Then, run the code below to slice each screenshot into its constituent sub images.

Continue with pixplot. The final cell should run pixplot in this notebook, but if the loading screen says 'NaN', stop the cell and just download the zipped output to your own machine. Assuming you have python installed, in a terminal run

```python -m http.server 5000```

then go to localhost:5000 in a browser on your machine. Examine this 'infographic'. What seems to define a cluster? What does this tell you about the underlying 'understanding' of the concept?

![](https://i.imgur.com/BWYnDJt.png)

_'The Battle of Vimy Ridge', according to Craiyon_

[Here is an example output using images generated with 'The Battle of Vimy Ridge' as a prompt](https://shawngraham.github.io/homecooked-history/genai-images-as-infographics/demo/) using the image generator at perchance.org/ai-text-to-image-generator, which clearly has a very different training set (or perhaps guardrails, lack thereof?)

[^1]: Modified. In the original post, there's a lot of close-reading of the image(s) going on. Here, we're also trying to use cnn to cluster images to cycle between close and distant reads.

In [ ]:
## run this
#!rm -r input #if you're processing different images w/ different aspect ratios, do it in batches. Do one batch, then uncomment this line to get a fresh input folder. Then change column/rows as appropriate, below
!mkdir input

In [ ]:
#uncomment this if you want to start over
#%cd ..
#!rm -r all_images

/content


### use this cell to unzip existing images:

In [ ]:
# if you have a folder of zipped images ready to go
# drag and drop the zipped folder into the file tray (hit the the folder icon to expand it if necessary) at left
# then adjust this code to use your file name, and run it:
!unzip my_images.zip -d input

## create some necessary functions for manipulating grids of images

You only need this if the images you uploaded and unzipped are in a grid, eg, like when you take a screenshot of craiyon.ai's grid of generated images. We want to feed one image at a time to pixplot for visualization, so we need to split the grid into nine separate images.

In [ ]:
import os
from PIL import Image
import math
import logging

def setup_logging():
    """Configure logging to track image processing details."""
    logging.basicConfig(level=logging.INFO,
                       format='%(asctime)s - %(levelname)s - %(message)s')
    return logging.getLogger(__name__)

def validate_dimensions(img_width, img_height, num_columns, num_rows):
    """
    Validate that the image can be evenly divided into the specified grid.

    Returns:
        tuple: (is_valid, single_width, single_height, warning_message)
    """
    single_width = img_width / num_columns
    single_height = img_height / num_rows

    # Check if dimensions result in whole numbers
    is_width_whole = single_width.is_integer()
    is_height_whole = single_height.is_integer()

    warning_msg = ""
    if not (is_width_whole and is_height_whole):
        warning_msg = (
            f"Warning: Image dimensions ({img_width}x{img_height}) "
            f"cannot be evenly divided into {num_columns}x{num_rows} grid. "
            f"Subimages will be {single_width:.2f}x{single_height:.2f} pixels."
        )

    return (is_width_whole and is_height_whole,
            int(single_width),
            int(single_height),
            warning_msg)

def slice_image(image_path, output_dir, num_columns=3, num_rows=2, strict_mode=True):
    """
    Slice an image into a grid of smaller images with validation and logging.

    Args:
        image_path (str): Path to the input image
        output_dir (str): Directory to save the output images
        num_columns (int): Number of columns in the grid
        num_rows (int): Number of rows in the grid
        strict_mode (bool): If True, raises error on uneven divisions

    Returns:
        list: List of paths to the generated images
    """
    logger = setup_logging()

    # Load the image
    img = Image.open(image_path).convert("RGB")
    img_width, img_height = img.size

    logger.info(f"Processing image: {image_path}")
    logger.info(f"Original dimensions: {img_width}x{img_height}")

    # Validate dimensions
    is_valid, single_width, single_height, warning_msg = validate_dimensions(
        img_width, img_height, num_columns, num_rows
    )

    if warning_msg:
        logger.warning(warning_msg)
        if strict_mode:
            raise ValueError("Image dimensions must be exactly divisible in strict mode")

    # Ensure we're working with integer dimensions
    single_width = math.floor(single_width)
    single_height = math.floor(single_height)

    logger.info(f"Subimage dimensions: {single_width}x{single_height}")

    output_paths = []
    for row in range(num_rows):
        for col in range(num_columns):
            left = col * single_width
            upper = row * single_height
            right = left + single_width
            lower = upper + single_height

            # Create a new blank image instead of cropping
            cropped_img = Image.new('RGB', (single_width, single_height))
            # Copy the exact region we want
            region = img.crop((left, upper, right, lower))
            cropped_img.paste(region, (0, 0))

            # Strip any existing metadata
            data = list(cropped_img.getdata())
            clean_img = Image.new('RGB', cropped_img.size)
            clean_img.putdata(data)

            base_name = os.path.splitext(os.path.basename(image_path))[0]
            original_ext = os.path.splitext(image_path)[1].lower()
            output_path = os.path.join(
                output_dir,
                f'{base_name}_cropped_{row * num_columns + col + 1}{original_ext}'
            )

            # Save with explicit dimensions
            clean_img.save(output_path, format=original_ext[1:])

            logger.info(f"Saved subimage: {output_path}")
            output_paths.append(output_path)

    return output_paths

def process_images(input_dir="input", output_dir="all_images",
                  num_columns=3, num_rows=2, strict_mode=True):
    """
    Process all supported image files in the input directory with validation.

    Args:
        input_dir (str): Input directory containing images
        output_dir (str): Directory to save the output images
        num_columns (int): Number of columns in the grid
        num_rows (int): Number of rows in the grid
        strict_mode (bool): If True, raises error on uneven divisions
    """
    logger = setup_logging()

    SUPPORTED_FORMATS = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
    os.makedirs(output_dir, exist_ok=True)

    processed_files = 0
    errors = 0

    for filename in os.listdir(input_dir):
        if filename.endswith(SUPPORTED_FORMATS):
            image_path = os.path.join(input_dir, filename)
            try:
                slice_image(image_path, output_dir, num_columns, num_rows, strict_mode)
                processed_files += 1
                logger.info(f"Successfully processed: {filename}")
            except Exception as e:
                errors += 1
                logger.error(f"Error processing {filename}: {str(e)}")

    logger.info(f"Processing complete. {processed_files} images processed, {errors} errors.")
    logger.info(f"Subimages saved to '{output_dir}' directory.")



## slice the images
If your image generator generates previews in a grid-like format, you can take a screenshot of that grid and then run 'process_images' below to cut them into individual images. Just change the number of columns and rows appropriately Eg, craiyon gives you 3x3 preview images;  https://perchance.org/ai-text-to-image-generator with 'casual photography', 6 photos, portrait, will return 3 x 2.

In [ ]:
process_images(
    input_dir="input",
    output_dir="all_images",
    num_columns=7,  #make sure you set this correctly!
    num_rows=4,  #make sure you set this correctly!
    strict_mode=False
)

# For strict validation (will raise error if dimensions don't divide evenly)
#process_images(strict_mode=True)

# For more lenient processing (will warn but continue)
#process_images(strict_mode=False)

## Get Pixplot Sorted Out

Because Pixplot was made for an earlier version of the python programming language, we have to do some shennanigans to make the right version of python available here. Then, whenever we want to run a pixplot command, we need to also tell Colab to use the earlier version.

In [ ]:
!pip install -q condacolab
import condacolab

condacolab.install()

✨🍰✨ Everything looks OK!


+ nb If you get some sort of message about crashing or kernel restarting, just carry on from here with the 'conda create' command below.

In [ ]:
%%capture
!conda create -n pixplot_test python=3.7

In [ ]:
#activate pixplot_test
!source activate pixplot_test; python --version; pip -V; pip install https://github.com/yaledhlab/pix-plot/archive/master.zip
!source activate pixplot_test; pip install ipykernel

In [ ]:
# make sure you're in the content directory
%pwd

# if you're not, use the cd command accordingly to get you there.

### here's the line that actually does the visual similarity analysis and creates the visualization

In [ ]:
### change the name of the folder between the quotes if you need to to point to your folder of images if you didn't use the slicer. KEEP the /* It means 'all the things inside'

!source activate pixplot_test; pixplot --images "all_images/*"

In [ ]:
#make sure you're in content
%pwd
#%cd content #change directory if necessary

## Download the Pixplot visualization

The next cell zips the pixplot 'output' folder. When it's finished, right-click on 'pixplot_visualization.zip' in your file tray and select download. It's a big file, it'll take a few moments.

Then on your own machine, unzip it (windows people, 'extract all'). Open a terminal or command prompt in the folder. I will assume you have python installed; start a server with

```python -m http.server 5000```

and go to https://localhost:5000 in your browser to load and explore the visualization.

In [ ]:

!zip -r pixplot_visualization.zip output

## then download it; unzip it
## and assuming you have python installed, in a terminal run
## python -m http.server 5000
## then go to localhost:5000 in a browser on your machine

### Or try running it here in colab


In [ ]:
# for reasons I can't figure out yet
# running pixplot from colab
# sometimes doesn't work and might
# fail at one crucial spot here, not loading a piece - check the log for 404 error; if you see that you'll know something went wrong.
# If it fails, that's a colab issue, not a *you* issue.
# IF THAT HAPPENS not to worry: zip it up, download, unzip, run the terminal or command prompt in that unzipped folder and run the python -m http.server 50000 command on your computer and it will work.

from google.colab import output
output.serve_kernel_port_as_iframe(5000)
%cd output
!python -m http.server 5000
